# 19107338 Computing Final assessment
## Physics engine for Unhappy Birds

### Introduction

In this task I will be creating a clone of the popular game angry birds using the Vpython module for python 3.

With this game I will attempt to simulate the physics of 2D motion under gravity and elastic collisions. The aim is to have a system in which a 'bird' is fired by the user at a target. If the user hits the target then the resulting torque is calculated. If this torque is greater than the restoring force then the target will topple and the user will win. Key sections of the code will be explained below.

### The game loops

My game is composed of 3 loops:

#### The 'go' loop:
This loop is the main loop in which most of the game code is enclosed. There is an input variable which allows to user to decide if they want to play the game or not. The start of this loop is returned to when the user has either hit or missed the target. 

#### The 'start' loop:
This is an introduction loop. The user is shown the game scene and asked for a value of v0 and theta. The user is then show the momentum vector these values will produce and given the option to continue or re-enter v0 and theta values. If the user choses to continue then the loop will be broken.

#### The 'run_game' loop:
This loop runs the animation of the bird undergoing 2D motion and contains a collision detection section. Once the bird has hit or missed the target the loop will break returning the user to the start of the go loop. 

### Functions used
I found this game had limited need for functions as very few lines of code were repeated. Only two functions were used:

#### The 'update_momentum' function:
This function takes in the user's values for v0 and theta and calculates the bird's momentum at time t. An arrow is then created with a start position on the surface of the bird and pointing in the direction of the bird's momentum. The length of the arrow is proportional to the magnitude of the bird's momentum. This function is used in both the intro section, to give the user a hint of the direction the bird will travel in, and the main animation section in which it is updated along with bird's position as time increases.

#### The 'update_end_text' function:
This function takes in a range of possible end game text strings and displays them to the user as label. This function is used for each possible outcome of the game: the bird misses, the bird hits but the target stays upright or the bird hits and the target topples.

### The physics
There are two main physics sections in this game both using equations from [2]:

#### 2D motion:
This section uses the equations shown in [2,pp.2]. The users inputs of v0 and theta are used to calculate the new position and momentum of the bird as time is increased by a set time step. These equations are continually implemented until the bird either hits the target or the ground.

#### Collision with the target:
This section uses the equations shown in [2,pp.3-4]. After the target is hit the applied force is calculated from the bird's momentum at this time. This collision point is assumed to be the bird's position at this time so the distance vector is the just the position of the bird minus the position of the target's bottom right hand corner. The applied torque is then calculated. If this is greater than the restoring torque then the target will topple and the user will win. The target is rotated by 90 degrees [1] to show it falling. If the target doesn't topple then the user is given the chance to play again.

### Collision detection

In order to detect collisions I had to check the birds position relative to the floor and the target. When checking for collisions with the floor I made it conditional that the birds y position was greater than 0.3. This meant that if the bottom of the ball hit the ground it would count as a collision. If the bird's x position was found to be greater than the distance to the target plus the target's width but was not at a height less then or equal to the targets height then the shot was treated as a miss.


For target collisions the condition was that the birds x position was greater than or equal to the target's distance and the birds y position was less than or equal to the targets height. This collision was included in an elif statement as it was possible both these statements were true but the bird had just overshot as mentioned above. 


### The import section

Here I have imported the required modules needed for the game to work.

In [3]:
import numpy as np
from vpython import sphere, color, rate, canvas, vector, curve, label, box, cross, mag, random, arrow, sleep #[1]

### The Game 

This section contains all the graphical and physical coded needed to run the game unhappy birds. I have broken the code into main sections which each have a different focus. 


In [ ]:
######################
###  Scene setup   ###
######################
#Creates the basic scene layout as given in [2]

scene = canvas(width=640, height=480, center=vector(8,5,0),range=8, userzoom = False)
ground = curve(pos=[(0,0,0),(16,0,0)],color=color.green)

###################################
###  Add objects to the scene  ####
###################################
#Creates the additional scene objets such as the target, bird, platform and text boxes

#Random values needed later
target_distance = np.random.uniform(5,15) #m
plat_height = np.random.uniform(0,1) #m

#The Target
target = box(pos=vector(target_distance,1,0), size=vector(0.5,2,0.5))
target.top = 2
target.left = target_distance-0.5

#The Platform
platform = box(pos=vector(0,0.5*plat_height,0), size=vector(0.5,plat_height,0.5), color = color.red)
M = 100 #kg

#The bird
bird = sphere(pos=vector(0,plat_height+0.3,0), radius=0.3, color = color.yellow, make_trail = True, trail_color = color.blue)
m = 0.1 #kg
x0 = 0 #m
y0 = plat_height+0.3 #m
momentum = arrow(pos=bird.pos, axis=vector(0,0,0), shaftwidth=0.05, color = color.red)


#Text boxes
Title_text = label( pos=vector(8,10,0), text='Unhappy Birds', color = color.yellow, box = False, height = 60 )
End_text = label( pos=vector(8,6,0), text='End text', color = color.green, box = False, height = 20, visible = False )
position_label = label(pos=vector(2,8,0), text='Position', color = color.white, line = False)
momentum_label = label(pos=vector(14,8,0), text='Momentum', color = color.white, line =False)

########################################
###  Constants needed for the game  ####
########################################

delta_t = 0.01 #s
time_step = 0.0001 #s
g = 9.81 #ms−2

########################################
###  Functions needed for the game  ####
########################################

def update_momentum(v0, theta, t, bird):
    """ 
    Calculates the x and y momentum components and creates an arrow based on these values
    pointing in the direction of the birds momentum and with a length equal to the magnitude
    of the momentum
    
    inputs: v0 - initial velocity of the bird
            theta - the launch angle of the bird
            t - current time
            bird - the bird object
    """
    p_x = m*v0*np.cos(theta) #kgms^-1
    p_y = m*v0*np.sin(theta) - m*g*t #kgms^-1
    
    momentum.pos = bird.pos
    momentum.axis = vector(p_x, p_y, 0)
    momentum.length = mag(momentum.axis)+0.3


def update_end_text(text, color):
    """ 
    updates the text shown after the bird has missed or hit the target
    
    inputs: text -  a string of text to be shown
            color - the color of the text to be displayed
    """
    
    End_text.color = color
    End_text.visible = True
    End_text.text = text

    
    

########################################
###            The Game             ####
########################################

#Prints a welcome message to the console
print('Welcome to unhappy birds! The best physics based game not on the market.')

#Sets the variable controling the main loop to true allowing the loop to start
go = True

#Starts the main loop
while go:
    rate(1000)
    
    #Stats the tutorial loop
    start = input('Do you want to play? y/n ')
    
    if start != 'y':
        print('Oh okay then, game ending... please still mark my work')
        break
    
    
####################################################
###                 Intro section                ###
### Where the user inputs values for theta and v0###
####################################################
    
    while start == 'y':
        
        #Starts time at zero
        t = 0
        
        #Initial graphical settings
        momentum.visible = False
        shown_1 = False
        shown_2 = False
        bird.clear_trail()
        bird.pos = vector(0,plat_height+0.3,0)
        End_text.visible = False
        scene.camera.pos = vector(8, 5, 13.8564) 
        
        #Redraws the scene
        scene.waitfor("redraw")
        
        #Optional graphical changes for the user
        follow = input('Would you like the camera to follow the bird as it moves? y/n: ')
        
        #User inputs values for theta and v0
        v0 = float(input('Please input the starting velocity of the bird in ms^-1: ')) #ms^-1
        theta = np.radians(float(input('Please input the starting angle of the bird in degrees: '))) #degrees
        
        #Shows the momentum arrow
        momentum.visible = True
        update_momentum(v0, theta, t, bird)
        
        #Redraws the scene
        scene.waitfor("redraw")
        
        #Asks user if they want the game to continue
        run_game = input('Are these your chosen values? y/n: ')
        
        #Begins the animation if the user is happy with their values
        if run_game == 'y':
            start = False
    
####################################################
###           Main animation section             ###
####################################################
#Here the flight of the bird is animated position and momentum values are updated
    
    while run_game == 'y':
        rate(1000)
        
        #Implement equations of motion
        r_x = x0 + v0*t*np.cos(theta) #m
        r_y = y0 + (v0*t*np.sin(theta) - ((g*(t**2))/2)) #m
        
        #Updates the birds position and momentum
        bird.pos = vector(r_x, r_y, 0)
        update_momentum(v0, theta, t, bird)
        
        #Keeps the user updated on momentum and position of the bird
        position_label.text = 'rx: {0:.1f}, ry: {1:.1f}'.format(r_x, r_y)
        momentum_label.text = 'px: {0:.1f}, py: {1:.1f}'.format(momentum.axis.x, momentum.axis.y)
        

        #Allows the camera to follow the path of the bird
        if follow == 'y':
            scene.camera.pos = bird.pos + vector(0,0,10)
            
            
        if t == 10 and shown_1 == False:
            print('Wow you really messed this up didn\'t you')
            shown_1 = True
            
        if t == 50 and shown_2 == False:
            print('If the bird doesn\'t land soon this computer will explode')
            shown_2 = True
            
        #Increases time by the time step
        t += time_step

####################################################
###              Collision detection             ###
####################################################
#Collisions with the ground and target are detected with the user being given an output

        #Checks if the bird has hit the ground or has overshot the target
        if bird.pos.y -0.3 <= 0 or bird.pos.x > target_distance+0.5:
            
            update_end_text('You missed! Maybe next time try and hit the target', color.red)
            
            #Checks if the bird has hit the ground
            if bird.pos.y-0.3 <= 0:
                
                #Outputs the distance travelled by the bird to the user
                print('Touch down! The bird landed {0:.2f} m away'.format(r_x))
                
                #breaks the animation loop
                run_game = False
                
            
        #Checks if the bird has hit the target
        elif bird.pos.x >= target_distance and bird.pos.y <= target.height:
            
            #Implements the equations for torque
            
            #Restoring torque
            mag_T_restoring = M*g*(0.5/2) #Nm
            
            #Applied torque
            F_applied = momentum.axis/delta_t #N
            collision_point = bird.pos
            d = collision_point - vector(target_distance,0,0) #m
            T_applied = cross(F_applied,d) #Nm
            mag_T_applied = mag(T_applied) #Nm
            
            if mag_T_applied > mag_T_restoring:
                
                #Rotates the target to show it fallen over [1]
                target.rotate(angle= -np.pi/2, axis=vector(0,0,1), origin=vector(target_distance,0,0))
                
                update_end_text('The target fell, you win!', color.green)
                
                #Outputs these print statments to the console
                print('The bird hit the target at height {0:.2f} m'.format(r_y))
                print('The momentum of the bird at the time of collision was ({0:.2f}, {1:.2f}, 0) kgms^-2'.format(momentum.axis.x, momentum.axis.y))
                print('The value of the applied torque was ({0:.2f}, {1:.2f}, {2:.2f}) Nm '.format(T_applied.x, T_applied.y, T_applied.z))
                print('The magnitude of the restoring torque was {0:.2f} Nm'.format(mag_T_restoring))
                
                #Redraws the scene
                scene.waitfor("redraw")
                
                sleep(5)
                
                #breaks the animation loop
                run_game = False
                
                #breaks the main game loop
                go = False
                
                
            
            else:
                update_end_text('The target didn\'t fall, try again!', color.red)
                
                #Redraws the scene
                scene.waitfor("redraw")
                
                sleep(5)
                
                #breaks the animation loop
                run_game = False   
                
                
                
print('Game over. Thank you very much for playing.')
    

<IPython.core.display.Javascript object>

Welcome to unhappy birds! The best physics based game not on the market.
Do you want to play? y/n y


### Assumptions and improvements 

As mentioned in the specification [2,pp.2-3] there are certain physical assumption made in order to simplify the simulation. However there are a few other assumptions made. There were also certain improvements which could be added to the game to increase the physical accuracy and game play.

* The contact point of the bird and the target is given by the birds x and y position. This means the bird acts as a point mass and can be seen clipping into the target once it has hit. This is because the simulation waits until the center of the bird collides with the wall. Initially I was going to add an offset of 0.3 in the x direction to this value to compensate for the birds radius. However, I then noticed that if the bird hit the top of the wall then this offset would need to be included in the y direction instead. A third condition would also be needed for if the bird hit the left hand corner of the target with an offset needing to be included in both directions. I therefore decided to keep the bird as a point mass as it worked best to show the physics of the system. However adding this in would increase the visuals of this game and make it more user friendly.


* The target can't wobble. In reality the target could return to its upright position and then topple slightly in the other direction. This would require the lower left corner of the target to also be considered as a point of rotation. This would increase the physical accuracy of the game and add more interesting game play scenarios. 


* A really interesting improvement would be to add a catapult system. This would allow the user to click and drag on the bird, pulling it back from its original position. The distance pulled back would be proportional to the initial velocity of the bird. While the negative value of the angle between the vector passing through the original position of the ball and the vector joining the new position of the bird to its original position would be theta. This would improve game play and make the controls more user friendly. 


* I considered putting restrictions on values of theta and v0. This would ensure a player can't fire the bird backwards or have such a high value of of v0 that the simulation fails to keep up or hitting the target becomes too easy. However, as the main function of this game is to act as a physical simulation I decided not to add any restrictions so the way in which the simulation acts under extreme values can be explored.

### References
* [1] Chabay R, Scherer D, and Sherwood B. 2017. https://www.glowscript.org/docs/VPythonDocs.html
* [2] Dash L. December 9 2019. PHAS0007: Final assignment 2019/20